<a href="https://colab.research.google.com/github/JoshStrother/DataVisualization/blob/main/Lesson5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Once again, we will just grab the first few rows of data to see what we are working with.

In [ ]:
%%bigquery --project data-viz-338716
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 5

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


I am still unsure of whats going on with the discrepancy between the station ID and station name so I will dig into that a little further.

In [ ]:
%%bigquery --project data-viz-338716
SELECT start_station_id, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
ORDER BY start_station_id ASC
LIMIT 5


,start_station_id,start_station_name
0,None,Zilker Park at Barton Springs & William Barton...
1,None,Zilker Park at Barton Springs & William Barton...
2,None,Main Office
3,None,Convention Center/ 3rd & Trinity
4,None,Convention Center/ 3rd & Trinity


Okay so it appears that some stations do not have an ID, also before limiting it I ran it as a full table of 20000 rows and there were stations with the same ID but different name.  

For the first part of this assignment, it is still feasible to use the station ID because we are just going to check how many times it appears in order to see the most 'popular'.

In [ ]:
%%bigquery --project data-viz-338716
SELECT start_station_id, COUNT(start_station_id) as num_of_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_id
ORDER BY COUNT(start_station_id) DESC

,start_station_id,num_of_trips
0,3798.0,83688
1,2575.0,49640
2,2707.0,43438
3,2499.0,40116
4,2494.0,39813
...,...,...
95,3456.0,359
96,4938.0,262
97,1005.0,250
98,1001.0,12


In [29]:
%%bigquery --project data-viz-338716
SELECT MAX(num_of_trips) as highest_trips
FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips
      FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
      GROUP BY start_station_id
      ORDER BY COUNT(start_station_id) DESC)

,highest_trips
0,83688


In [48]:
%%bigquery --project data-viz-338716
SELECT MAX(num_of_trips) as highest_trips
FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips
      FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
      GROUP BY start_station_id
      ORDER BY COUNT(start_station_id) DESC)
WHERE num_of_trips != (SELECT MAX(num_of_trips) as highest_trips FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` GROUP BY start_station_id ORDER BY COUNT(start_station_id) DESC ))

,highest_trips
0,49640


After attempting many time to include the start station ID, I could not get it to work, but we know that the second most popular station had 49640 trips.  If we look above at the listed order we know that the station ID is 2575.

THE FOLLOWING CODE IS ONE OF MY MANY ATTEMPTS TO INCLUDE THE STATION ID

In [66]:
%%bigquery --project data-viz-338716
SELECT start_station_id, MAX(num_of_trips) as highest_trips
FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips
      FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
      GROUP BY start_station_id
      ORDER BY COUNT(start_station_id) DESC)
WHERE num_of_trips != (SELECT MAX(num_of_trips) as highest_trips FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` GROUP BY start_station_id ORDER BY COUNT(start_station_id) DESC ))
GROUP BY start_station_id
ORDER BY COUNT(start_station_id) DESC

Executing query with job ID: 8e9dc9b1-43ff-4f3e-abe0-c70d8a0c865c
Query executing: 0.27s


ERROR:
 400 SELECT list expression references column num_of_trips which is neither grouped nor aggregated at [1:26]

(job ID: 8e9dc9b1-43ff-4f3e-abe0-c70d8a0c865c)

                                                                                                                      -----Query Job SQL Follows-----                                                                                                                       

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT start_station_id, num_of_trips
   2:FROM (SELECT start_station_id, COUNT(start_station_id) as num_of_trips
   3:      FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
   4:      GROUP BY start_station_id
   5:      ORDER BY COUNT(start_station_id) DESC)
   6:WHERE num_of_tr

__________________________________________________________________________________________________________

I am going to attempt to gather the trips that start/ended at the same place and that lasted longer than an hour (60 mins)

In [76]:
%%bigquery --project data-viz-338716
SELECT trip_id, start_station_name, end_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name AND duration_minutes > 60

,trip_id,start_station_name,end_station_name,duration_minutes
0,9900290119,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,453
1,9900294107,11th & San Jacinto,11th & San Jacinto,148
2,9900294112,ACC - West & 12th Street,ACC - West & 12th Street,173
3,9900294318,11th & San Jacinto,11th & San Jacinto,147
4,9900294483,Nueces @ 3rd,Nueces @ 3rd,365
...,...,...,...,...
54790,9198253,3rd & West,3rd & West,127
54791,15686574,3rd & West,3rd & West,127
54792,22065516,3rd/West,3rd/West,127
54793,20489144,3rd/West,3rd/West,127


Okay so that seemed to be relatively easy, now im going to try to use that as a sub-query to count that information and see how many trips there are...obviously as of right now we can see there are 54794, but that isnt what the assignment asked us to do :)

In [75]:
%%bigquery --project data-viz-338716
SELECT COUNT(trips) as num_trips_startstop_over_hour
FROM (SELECT trip_id as trips, start_station_name, end_station_name, duration_minutes
      FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
      WHERE start_station_name = end_station_name AND duration_minutes > 60)

,num_trips_startstop_over_hour
0,54795


So this isn't exactly how I pictured I would end up doing it, but it works and its correct so there that.